In [1]:
# pip install all the dependencies
!pip install openai


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
# Setup and Installation
import os
from openai import OpenAI
import json
from pprint import pprint

# First, let's install the required packages
# !pip install openai

# Initialize the Fireworks client
client = OpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY", "Enter you API key here")
)

print("✅ Fireworks client initialized!")
print(f"Base URL: {client.base_url}")

# Verify we can connect to the API
try:
    models = client.models.list()
    print(f"✅ Successfully connected to Fireworks API!")
    print(f"Found {len(models.data)} available models")
except Exception as e:
    print(f"❌ Error connecting to Fireworks API: {e}")
    print("Please check your FIREWORKS_API_KEY environment variable")

✅ Fireworks client initialized!
Base URL: https://api.fireworks.ai/inference/v1/
❌ Error connecting to Fireworks API: Error code: 403 - {'error': 'unauthorized'}
Please check your FIREWORKS_API_KEY environment variable


In [3]:
# First call: ask for 5 interesting facts
print("🔍 First Query: 5 interesting facts with store as False, meaning wont be stored in Redis DB")
print("=" * 60)

first_stream = client.responses.create(
    model="accounts/fireworks/models/qwen3-235b-a22b",
    input="give me 5 interesting facts on modelcontextprotocol/python-sdk -- keep it short!",
    stream=True,
    store=False,
    tools=[{"type": "mcp", "server_url": "https://mcp.deepwiki.com/mcp"}]
)

# Print each event object as it arrives
first_response_id = None

for chunk in first_stream:
    print(chunk)  # This will print the full event object
    
    # Still capture response ID for follow-up
    if hasattr(chunk, 'response') and chunk.response and chunk.response.id:
        first_response_id = chunk.response.id

print("=" * 60)

🔍 First Query: 5 interesting facts with store as False, meaning wont be stored in Redis DB
ResponseCreatedEvent(response=Response(id='resp_7d9550f48239410e826e1cb126d8d072', created_at=1750218114.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='accounts/fireworks/models/qwen3-235b-a22b', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[Mcp(server_label=None, server_url='https://mcp.deepwiki.com/mcp', type='mcp', allowed_tools=None, headers=None, require_approval=None)], top_p=1.0, background=None, max_output_tokens=None, previous_response_id=None, reasoning=None, service_tier=None, status='in_progress', text=None, truncation='disabled', usage=None, user=None, store=False), sequence_number=0, type='response.created')
ResponseInProgressEvent(response=Response(id='resp_7d9550f48239410e826e1cb126d8d072', created_at=1750218114.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='accoun

In [5]:
# Second call: ask to explain the second fact in more detail
if first_response_id:
    print("\n🔍 Follow-up Query: Explain second fact in more detail, this will fail because we set Store as False in previous API Call, and the previous response id wont be store")
    print("=" * 60)
    
    second_stream = client.responses.create(
        model="accounts/fireworks/models/qwen3-235b-a22b",
        input="Explain second fact in more detail",
        previous_response_id=first_response_id,
        stream=True,
        tools=[{"type": "mcp", "server_url": "https://mcp.deepwiki.com/mcp"}]
    )

    # Print each event object as it arrives
    for chunk in second_stream:
        print(chunk)  # This will print the full event object
    
    print("=" * 60)


🔍 Follow-up Query: Explain second fact in more detail, this will fail because we set Store as False in previous API Call, and the previous response id wont be store


NotFoundError: Error code: 404 - {'detail': 'Previous response not found'}